In [1]:
import functools

In [2]:
import numpy as np

In [3]:
import math

In [4]:
import tensorflow as tf

In [5]:
from tensorflow.python import debug as tf_debug

In [6]:
from enum import IntEnum

In [7]:
print("TensorFlow version: {}".format(tf.VERSION))

TensorFlow version: 1.8.0


In [8]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
training_data, testing_data = tf.keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 9s 2us/step


In [9]:
number_of_classes = np.max(testing_data[1]) + 1
number_of_input_channels = 1

In [10]:
batch_size = 32

In [11]:
number_of_layers = 3 # 1000

In [12]:
lod_channels = [32, 16, 8]

In [13]:
def get_number_of_samples_per_fiber():
    return 3

In [14]:
OffsetIndex = IntEnum("OffsetIndex", names=["OFFSET_X", "OFFSET_Y"], start=0)
ChannelIndex = IntEnum("ChannelIndex", names=["TARGET_CHANNEL", "SOURCE_CHANNEL"], start=0)

In [15]:
root_variable_scope = tf.get_variable_scope()

In [16]:
def fiber(number_of_target_channels, number_of_source_channels, source_layer):
#     number_of_source_channels = source_layer[3]
    def generate_channel_indices():
        for target_channel_index in range(number_of_target_channels):
            for source_channel_index in range(number_of_source_channels):
                number_of_samples_per_fiber = get_number_of_samples_per_fiber()
                for fiber_index in range(number_of_samples_per_fiber):
                    yield (target_channel_index, source_channel_index)
    channel_indices = tf.get_variable(name="channel_indices", initializer=tuple(generate_channel_indices()), trainable=False)
    target_channel_indices, source_channel_indices = tf.unstack(channel_indices, axis=1)
    number_of_sample_points, number_of_channel_indices = channel_indices.shape
    assert number_of_channel_indices == len(ChannelIndex.__members__)
    offset_xy = tf.get_variable(name="offset_xy", dtype=tf.float32, shape=(number_of_sample_points, len(OffsetIndex.__members__)))
    weight = tf.get_variable(name="weight", dtype=tf.float32, shape=[number_of_sample_points, 1, 1, 1])
    return tf.transpose(
        tf.unsorted_segment_sum(
            weight * tf.contrib.image.translate(
                tf.gather(
                    tf.transpose(
                        source_layer,
                        perm=(3, 1, 2, 0)    
                    ),
                    indices=source_channel_indices
                ),
                translations=offset_xy
            ),
            segment_ids=target_channel_indices,
            num_segments=number_of_target_channels
        ),
        perm=(3, 1, 2, 0)  
    )

In [17]:
def model_fn(features, labels, mode, params, config):
    input_shape = features.shape
    def lod_size(i):
        scale = 2 ** i
        return input_shape[1] // tf.Dimension(scale), input_shape[2] // tf.Dimension(scale)

    @functools.lru_cache(maxsize=None)
    def input_layer(current_lod):
        with tf.variable_scope(root_variable_scope), tf.variable_scope(f"scale_{current_lod}"), tf.name_scope(f"scale_{current_lod}/"):
            if current_lod == 0:
                return tf.expand_dims(tf.cast(features, tf.float32), axis=3) / 255.0
            else:
                # Pooling
    #             tf.nn.avg_pool(
    #                 value=layer(i, lod, current_lod - 1, channel),
    #                 ksize=[1, 2, 2, 1],
    #                 strides=[1, 1, 1, 1],
    #                 padding='SAME'
    #             )
                return tf.image.resize_bilinear(
                    images=input_layer(current_lod - 1),
                    size=lod_size(current_lod),
                    align_corners=False,
                )
    @functools.lru_cache(maxsize=None)
    def layer(target_layer_index, preferred_lod, current_lod):
        """Return a list of tensor of shape batch_size ⨉ lod_size(height) ⨉ lod_size(width) ⨉ channel

        The list size is sum(lod_channels)
        """
        with tf.variable_scope(root_variable_scope), tf.variable_scope(f"layer_{target_layer_index}"), tf.name_scope(f"layer_{target_layer_index}/"):
            if current_lod == preferred_lod:
                number_of_target_channels = lod_channels[preferred_lod]
                with tf.variable_scope(f"lod_{preferred_lod}"):
                    def source_layer_mapper(source_layer_index):
                        with tf.variable_scope(f"weighted_layer_{source_layer_index}"):
                            def source_lod_mapper(source_lod, number_of_source_channels):
                                with tf.variable_scope(f"weighted_lod_{source_lod}"):
                                    return fiber(
                                        number_of_target_channels=number_of_target_channels,
                                        number_of_source_channels=number_of_source_channels,
                                        source_layer=layer(source_layer_index, source_lod, current_lod)
                                    )
                            return sum(source_lod_mapper(source_lod, number_of_source_channels)
                                       for source_lod, number_of_source_channels in enumerate(lod_channels))
                    channel_scores = sum(map(source_layer_mapper, range(target_layer_index)))
                    bias = tf.get_variable(name="bias", dtype=tf.float32, shape=(1, 1, 1, number_of_target_channels))
                    with tf.variable_scope("weighted_input"):
                        input_fiber = fiber(number_of_target_channels=number_of_target_channels,
                                            number_of_source_channels=number_of_input_channels,
                                            source_layer=input_layer(current_lod))
                        return bias + input_fiber + channel_scores
            elif preferred_lod < current_lod:
                # Pooling
    #             tf.nn.avg_pool(
    #                 value=layer(i, lod, current_lod - 1, channel),
    #                 ksize=[1, 2, 2, 1],
    #                 strides=[1, 1, 1, 1],
    #                 padding='SAME'
    #             )
                return tf.image.resize_bilinear(
                    images=layer(target_layer_index, preferred_lod, current_lod - 1),
                    size=lod_size(current_lod),
                    align_corners=False,
                )
            elif preferred_lod > current_lod:
                # Unpooling
                return tf.image.resize_bilinear(
                    images=layer(target_layer_index, preferred_lod, current_lod + 1),
                    size=lod_size(current_lod),
                    align_corners=False,
                )
    lowest_lod = len(lod_channels) - 1
    scores = tf.nn.xw_plus_b(
        x=tf.reduce_mean(layer(number_of_layers - 1, lowest_lod, lowest_lod), axis=(1, 2)),
        weights=tf.get_variable(name="dense_weight", dtype=tf.float32, shape=(lod_channels[lowest_lod], number_of_classes)),
        biases=tf.get_variable(name="dense_bias", dtype=tf.float32, shape=number_of_classes)
    )
    probabilities = tf.nn.softmax(logits=scores)
    predictions = {
        'probabilities' : probabilities,
        'scores': scores
    }
    loss = tf.losses.softmax_cross_entropy(logits=scores, onehot_labels=tf.one_hot(labels, number_of_classes))
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions,
        loss=loss,
        train_op=train_op
    )


estimator = tf.estimator.Estimator(model_fn,model_dir=f"models/offnet{number_of_layers}")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/offnet3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdb1e52dcf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [18]:
def training_dataset():
    return tf.data.Dataset.from_tensor_slices(training_data).shuffle(1000).repeat().batch(batch_size)

In [ ]:
# hook = tf_debug.TensorBoardDebugHook("localhost:6064")
# estimator.train(training_dataset,hooks=[hook])
estimator.train(training_dataset)

INFO:tensorflow:Calling model_fn.


/mnt/project-root/envs/gpu/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into models/offnet3/model.ckpt.
INFO:tensorflow:loss = 2.4156346, step = 0
INFO:tensorflow:global_step/sec: 11.1882
INFO:tensorflow:loss = 2.1430933, step = 100 (8.940 sec)
INFO:tensorflow:global_step/sec: 12.64
INFO:tensorflow:loss = 2.0574737, step = 200 (7.911 sec)
INFO:tensorflow:global_step/sec: 12.606
INFO:tensorflow:loss = 2.2241282, step = 300 (7.933 sec)
INFO:tensorflow:global_step/sec: 12.5326
INFO:tensorflow:loss = 2.0664253, step = 400 (7.979 sec)
INFO:tensorflow:global_step/sec: 12.5711
INFO:tensorflow:loss = 1.9910045, step = 500 (7.955 sec)
INFO:tensorflow:global_step/sec: 12.4824
INFO:tensorflow:loss = 2.1563253, step = 600 (8.014 sec)
INFO:tensorflow:global_step/sec: 12.4898
INFO:tensorflow:loss = 1.9782343, step = 700 (8.003 sec

INFO:tensorflow:loss = 1.8302993, step = 8000 (8.634 sec)
INFO:tensorflow:global_step/sec: 12.2085
INFO:tensorflow:loss = 1.9298606, step = 8100 (8.191 sec)
INFO:tensorflow:global_step/sec: 11.8492
INFO:tensorflow:loss = 1.8426367, step = 8200 (8.439 sec)
INFO:tensorflow:global_step/sec: 12.2003
INFO:tensorflow:loss = 1.8546567, step = 8300 (8.197 sec)
INFO:tensorflow:global_step/sec: 12.2071
INFO:tensorflow:loss = 1.99364, step = 8400 (8.192 sec)
INFO:tensorflow:global_step/sec: 12.2052
INFO:tensorflow:loss = 2.0496435, step = 8500 (8.193 sec)
INFO:tensorflow:global_step/sec: 11.67
INFO:tensorflow:loss = 1.9956946, step = 8600 (8.569 sec)
INFO:tensorflow:global_step/sec: 11.5842
INFO:tensorflow:loss = 1.742407, step = 8700 (8.633 sec)
INFO:tensorflow:global_step/sec: 12.2014
INFO:tensorflow:loss = 1.9869413, step = 8800 (8.196 sec)
INFO:tensorflow:global_step/sec: 11.8077
INFO:tensorflow:loss = 2.0618086, step = 8900 (8.470 sec)
INFO:tensorflow:global_step/sec: 11.6716
INFO:tensorflow

INFO:tensorflow:loss = 2.0066416, step = 16200 (8.200 sec)
INFO:tensorflow:global_step/sec: 12.0616
INFO:tensorflow:loss = 2.0821905, step = 16300 (8.290 sec)
INFO:tensorflow:global_step/sec: 11.8792
INFO:tensorflow:loss = 2.1783009, step = 16400 (8.418 sec)
INFO:tensorflow:global_step/sec: 12.2205
INFO:tensorflow:loss = 2.304245, step = 16500 (8.183 sec)
INFO:tensorflow:global_step/sec: 12.2104
INFO:tensorflow:loss = 2.072259, step = 16600 (8.190 sec)
INFO:tensorflow:global_step/sec: 12.1788
INFO:tensorflow:loss = 1.8812071, step = 16700 (8.210 sec)
INFO:tensorflow:global_step/sec: 12.1381
INFO:tensorflow:loss = 1.7955763, step = 16800 (8.238 sec)
INFO:tensorflow:global_step/sec: 12.1634
INFO:tensorflow:loss = 1.8466775, step = 16900 (8.222 sec)
INFO:tensorflow:global_step/sec: 12.2075
INFO:tensorflow:loss = 1.9739169, step = 17000 (8.192 sec)
INFO:tensorflow:global_step/sec: 12.013
INFO:tensorflow:loss = 1.9961984, step = 17100 (8.324 sec)
INFO:tensorflow:global_step/sec: 12.2196
INF